```json
{
    "mask_name": "mask_00123.npy",
    "mask_height": 672,
    "mask_width": 630,
    "promote_type": "mask",
    "labels": {
        "3": {
            "instance_id": 3,
            "class_name": "table",
            "x1": 156,
            "y1": 333,
            "x2": 299,
            "y2": 414,
            "logit": 0.0
        },
        "5": {
            "instance_id": 5,
            "class_name": "table",
            "x1": 268,
            "y1": 463,
            "x2": 447,
            "y2": 666,
            "logit": 0.0
        }
    }
}
```

In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

def analyze_npy_files(directory):
    directory = Path(directory)
    npy_files = list(directory.glob("*.npy"))
    summary = {}
    
    for npy_file in npy_files:
        mask = np.load(npy_file)
        
        # Get unique values and their counts
        unique_values, counts = np.unique(mask, return_counts=True)
        
        summary[npy_file.name] = {
            "shape": mask.shape,
            "dtype": mask.dtype,
            "unique_values": unique_values.tolist(),
            "counts": counts.tolist()
        }
        
        # Display histogram of values
        plt.figure()
        plt.hist(mask.flatten(), bins=len(unique_values), edgecolor='black')
        plt.title(f"Value Distribution in {npy_file.name}")
        plt.xlabel("Mask Value")
        plt.ylabel("Count")
        plt.show()
    
    return summary

# Example usage
directory = Path("path_to_npy_files")
summaries = analyze_npy_files(directory)

# Print summary for review
for file, info in summaries.items():
    print(f"File: {file}")
    print(f" - Shape: {info['shape']}")
    print(f" - Data Type: {info['dtype']}")
    print(f" - Unique Values: {info['unique_values']}")
    print(f" - Counts: {info['counts']}")
    print()


In [ ]:
analyze_npy_files("/tmp/mask_data")